In [1]:
import collections.abc
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import time

import pandas as pd
import os
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import sys

# soup error 방지
import collections
if not hasattr(collections, 'Callable'):
    collections.Callable = collections.abc.Callable

C:\Users\LG\AppData\Local\Temp\ipykernel_2608\1179179661.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if not hasattr(collections, 'Callable'):


In [3]:
# 데이터 1만개 크롤링
for i in range(1,1000):  
    url =   f'https://kin.naver.com/search/list.naver?query=%EC%9D%B4%EB%B9%84%EC%9D%B8%ED%9B%84%EA%B3%BC&page={i}'
    html   =   urlopen(url)
    soup   =   BeautifulSoup(html.read(),   'html.parser')
    links = soup.find('ul', class_='basic1').find_all('a')


    links_list = []
    for link in links:
        if ('https://kin.naver.com/qna/detail.naver?d1id' in link['href']) and (link['href'] not in links_list):
            links_list.append(link['href'])

    # links_list = [link['href'] for link in links if 'https://kin.naver.com/qna/detail.naver?d1id' in link['href']]

    # 링크 출력 (최대 10개)
    # for href in links_list:
    #     print(f"{href}")

    for idx, url in enumerate(links_list):
        response = requests.get(url)

        # HTML을 BeautifulSoup으로 파싱
        soup = BeautifulSoup(response.text, 'html.parser')
        soup = soup.find('div', {'class':'questionDetail'})
        soup = soup.get_text(separator=' ').strip()
        FILE_PATH = '../../../LocalData/nlp/pj_data/'
        f = open(FILE_PATH+f"{i}, {idx}.txt", 'w')
        # print(i, idx, soup)
        f.write(soup)

FileNotFoundError: [Errno 2] No such file or directory: '../../../LocalData/NLP_DATA/pj_data/1, 0.txt'

In [5]:
textDF = pd.DataFrame(columns=['txt','label'])

# 본인 데이터 파일 지정
TRAIN_PATH = '../../../LocalData/nlp/otolaryngology_data/'
files = os.listdir(TRAIN_PATH)
idx = 0
for file in files:
    with open(TRAIN_PATH+file, mode='r', encoding='utf-8') as f:
        text=f.read()
        text = re.sub('[^가-힣]+',' ',text)
        textDF.loc[idx] = [text,1]
        idx+=1
        print(idx)
        # 전체 인덱스 지정용 idx

# 내과
i_idx=1
TRAIN_PATH = '../../../LocalData/nlp/medicine_data/'
files = os.listdir(TRAIN_PATH)
for file in files:
    with open(TRAIN_PATH+file, mode='r', encoding='utf-8') as f:
        text=f.read()
        text = re.sub('[^가-힣]+',' ',text)
        textDF.loc[idx] = [text,0]
        i_idx+=1
        idx+=1
        print(f"idx : {idx}, n_idx : {i_idx}")
        if i_idx == 3300: break
        # i_idx 는 inner_data용 idx 3300개만 뽑기 위한 용도, 아래 n_idx, m_idx 또한 같은 용도. 참고

# 정신과용
m_idx=1
TRAIN_PATH = '../../../LocalData/nlp/psychiatry_data/'
files = os.listdir(TRAIN_PATH)
for file in files:
    with open(TRAIN_PATH+file, mode='r', encoding='utf-8') as f:
        text=f.read()
        text = re.sub('[^가-힣]+',' ',text)
        textDF.loc[idx] = [text,0]
        m_idx+=1
        idx+=1
        print(f"idx : {idx}, n_idx : {m_idx}")
        if m_idx == 3300: break


# 안과용
n_idx=1
TRAIN_PATH = '../../../LocalData/nlp/eyes_data/'
files = os.listdir(TRAIN_PATH)
for file in files:
    with open(TRAIN_PATH+file, mode='r', encoding='utf-8') as f:
        text=f.read()
        text = re.sub('[^가-힣]+',' ',text)
        textDF.loc[idx] = [text,0]
        n_idx+=1
        idx+=1
        print(f"idx : {idx}, n_idx : {n_idx}")
        if n_idx == 3300: break


# 종료시 알림
import winsound as sd
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 300     # 1000 ms ==1second
    sd.Beep(fr, du)
beepsound()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [6]:
# 문장 분류 모델
from torch import nn

class SentenceClassifier(nn.Module):
    def __init__(
            self,
            n_vocab,
            hidden_dim,
            embedding_dim,
            n_layers,
            dropout=0.5,
            bidirectional=True,
            model_type="lstm"
    ):
        super().__init__()

        self.embedding=nn.Embedding(
            num_embeddings=n_vocab,
            embedding_dim=embedding_dim,
            padding_idx=0
        )
        if model_type =="rnn":
            self.model=nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        elif model_type =="lstm":
            self.model=nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional=bidirectional,
                dropout=dropout,
                batch_first=True,
            )
        if bidirectional:
            self.classifier=nn.Linear(hidden_dim * 2,1)
        else:
            self.classifier=nn.Linear(hidden_dim, 1)
        self.dropout=nn.Dropout(dropout)
    
    def forward(self, inputs):
        embeddings=self.embedding(inputs)
        output, _=self.model(embeddings)
        last_output=output[:, -1, :]
        last_output=self.dropout(last_output)
        logits=self.classifier(last_output)
        return logits

In [7]:
# 데이터세트 불러오기
train_data=textDF.sample(frac=0.9, random_state=42)
test_data=textDF.drop(train_data.index)

print(train_data.head(5).to_markdown())
print("Training Data Size :", len(train_data))
print("Testing Data Size :", len(test_data))

|       | txt                                                                                                                                                                                                  |   label |
|------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------:|
|  1430 | 이비인후과 집 가까이 있는곳 가려는데 팔달시장에 수이비인후과 소아청소년만 보나요                                                                                                                     |       1 |
| 16400 | 고 학생인데 중학생때부터 집중도 통 안되고 망상에 빠지는 경우가 너무 많아서 검사를 한 번 해보려고 하는데요 검색해보니 판정받으면 정신과 기록이 남는다 하던데 혹시 이게 나중에 악영향을 끼칠 수 있나요 |       0 |
|  8144 | 후비루 치료 잘하는 이비인후과                                                                                                                                                                        |       1 |
|  4610 | 이비인후과 집 가까이 있는곳 가려는데 팔달시장에 수

In [8]:
# 데이터 토큰화 및 단어 사전 구축
from konlpy.tag import Okt
from collections import Counter

def build_vocab(corpus, n_vocab, special_tokens):
    counter=Counter()
    for tokens in corpus:
        counter.update(tokens)
    vocab=special_tokens
    for token, count in counter.most_common(n_vocab):
        vocab.append(token)
    return vocab

tokenizer=Okt()
train_tokens=[tokenizer.morphs(review) for review in train_data.txt]
test_tokens=[tokenizer.morphs(review) for review in test_data.txt]

vocab=build_vocab(corpus=train_tokens, n_vocab=5000, special_tokens=["<pad>", "<unk>"])
token_to_id={token: idx for idx, token in enumerate(vocab)}
id_to_token={idx: token for idx, token in enumerate(vocab)}

print(vocab[:10])
print(len(vocab))

['<pad>', '<unk>', '이', '에', '가', '이비인후과', '를', '을', '는', '도']
5002


In [56]:
# 단어 사전 csv 저장
vocab_df = pd.DataFrame(columns=['key', 'value'])
vocab_df['key'] = token_to_id.keys()
vocab_df['value'] = token_to_id.values()
vocab_df.to_csv('../../../LocalData/nlp/my_vocab/nose_vocab.csv', encoding='utf-8')

In [9]:
# 정수 인코딩 및 패딩
import numpy as np

def pad_sequences(sequences, max_length, pad_value):
    result=list()
    for sequence in sequences:
        sequence=sequence[:max_length]
        pad_length=max_length-len(sequence)
        padded_sequence=sequence+[pad_value] * pad_length
        result.append(padded_sequence)
    return np.asarray(result)

unk_id=token_to_id['<unk>']
train_ids=[
    [token_to_id.get(token, unk_id) for token in review] for review in train_tokens
]
test_ids=[
    [token_to_id.get(token, unk_id) for token in review] for review in test_tokens
]

max_length=32
pad_id=token_to_id["<pad>"]
train_ids=pad_sequences(train_ids, max_length, pad_id)
test_ids=pad_sequences(test_ids, max_length, pad_id)

print(train_ids[0])
print(test_ids[0])

[  5 209 391 146 143 101 344 137 407   3  30   5 180 264  51 392  13   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
[ 850 2695    3 1097    2    9  391    1  195 1021 1791    1  793    5
   44 1053    9 1280 2745  147    1    1   68   32  508    1 2116 2052
 2869    1  196 1403]


In [10]:
# 데이터로더 적용
import torch
from torch.utils.data import TensorDataset, DataLoader

train_ids=torch.tensor(train_ids)
test_ids=torch.tensor(test_ids)

train_labels=torch.tensor(train_data.label.values, dtype=torch.float32)
test_labels=torch.tensor(test_data.label.values, dtype=torch.float32)

train_dataset=TensorDataset(train_ids, train_labels)
test_dataset=TensorDataset(test_ids, test_labels)

train_loader=DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader=DataLoader(test_dataset, batch_size=16, shuffle=False)

In [11]:
# 손실 함수와 최적화 함수 정의
from torch import optim

n_vocab=len(token_to_id)
hidden_dim=64
embedding_dim=128
n_layers=2

device="cuda" if torch.cuda.is_available() else "cpu"
classifier=SentenceClassifier(
    n_vocab=n_vocab, hidden_dim=hidden_dim, embedding_dim=embedding_dim, n_layers=n_layers
).to(device)
criterion=nn.BCEWithLogitsLoss().to(device)
optimizer=optim.RMSprop(classifier.parameters(), lr=0.001)

In [12]:
# 모델 학습 함수
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses=list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids=input_ids.to(device)
        labels=labels.to(device).unsqueeze(1)

        logits=model(input_ids)
        loss=criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval==0:
            print(f"Train Loss {step} : {np.mean(losses)}")

In [13]:
# 모델 테스트 함수
def test(model, datasets, criterion, device, min_loss, max_score, trigger, epoch):
    model.eval()
    losses = list()
    corrects = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())
        yhat = torch.sigmoid(logits)>.5
        corrects.extend(
            torch.eq(yhat, labels).cpu().tolist()
        )
    t_loss = np.mean(losses)
    t_score = np.mean(corrects)

    SAVE_FILE = f'../../../LocalData/nlp/models/model_train_wbs_{epoch}_{t_score}.pth' 
    SAVE_MODEL = f'../../../LocalData/nlp/models/model_train_all_{epoch}_{t_score}.pth' 

    if min_loss < t_loss:
        trigger+=1
        print('loss 상승')
    else : 
        trigger=0
        min_loss = t_loss
    
    if max_score < t_score:
        torch.save(model.state_dict() ,SAVE_FILE)
        torch.save(model, SAVE_MODEL)
    print(f"Val Loss : {t_loss}, Val Accuracy : {t_score}")
    

In [15]:
epochs=5
interval=500

min_loss, max_score, trigger=100, -1, 0

for epoch in range(epochs):
    train(classifier, train_loader, criterion, optimizer, device, interval)
    test(classifier, test_loader, criterion, device, min_loss, max_score, trigger, epoch)

Train Loss 0 : 0.0018088812939822674
Train Loss 500 : 0.02367465771524888
Train Loss 1000 : 0.026677782061135538
Val Loss : 0.04609592062467709, Val Accuracy : 0.9849170437405732
Train Loss 0 : 0.0021200955379754305
Train Loss 500 : 0.011666693229229653
Train Loss 1000 : 0.01521002236195118
Val Loss : 0.04352218983595958, Val Accuracy : 0.9879336349924586
Train Loss 0 : 0.0006782819982618093
Train Loss 500 : 0.010359467015352598
Train Loss 1000 : 0.011516225617583926
Val Loss : 0.02994807536894223, Val Accuracy : 0.9879336349924586
Train Loss 0 : 0.0010561657836660743
Train Loss 500 : 0.007407614347856818
Train Loss 1000 : 0.009227643865227226
Val Loss : 0.05551533526700223, Val Accuracy : 0.9894419306184012
Train Loss 0 : 5.547011096496135e-05
Train Loss 500 : 0.006981665550469217
Train Loss 1000 : 0.0061880764376249
Val Loss : 0.06203641538585362, Val Accuracy : 0.9889391654097537


In [16]:
# 학습된 모델로부터 임베딩 추출
token_to_embedding=dict()
embedding_matrix=classifier.embedding.weight.detach().cpu().numpy()

for word, emb in zip(vocab, embedding_matrix):
    token_to_embedding[word]=emb

token=vocab[1000]
print(token, token_to_embedding[token])

스펜서 [ 3.43262017e-01  8.12775791e-01 -1.47426856e+00 -6.38971567e-01
 -7.73742437e-01 -7.07819045e-01 -1.67777455e+00 -1.42115033e+00
 -1.20716500e+00 -8.35588574e-02  5.79760671e-01 -7.28973806e-01
  1.06186338e-01  1.41254628e+00 -5.82401514e-01  1.19523549e+00
 -9.02565479e-01 -4.04843241e-01 -6.15499496e-01 -3.08270872e-01
  9.26977634e-01  1.81862190e-01  1.25222397e+00  8.43519509e-01
 -4.72006172e-01 -2.03272557e+00 -9.50266063e-01  1.06136866e-01
 -4.77085680e-01  1.77455795e+00 -1.65660846e+00 -1.27701974e+00
  4.88800317e-01  2.02552342e+00  1.19815037e-01  9.16696787e-01
  3.60379666e-02  3.78756464e-01  5.36129549e-02  2.02428669e-01
 -2.22380817e-01  6.02795631e-02  1.11404848e+00  3.89376193e-01
  1.13675997e-01 -3.77129555e-01 -8.90802622e-01 -1.19903159e+00
  1.56371105e+00 -1.69291288e-01  1.39353240e+00  6.36655509e-01
  1.15345299e+00 -1.97222698e+00 -2.95266420e-01 -6.42524958e-01
 -6.32146358e-01 -6.25878930e-01 -3.03204060e-01  2.54847407e+00
 -2.21256423e+00 -1.2

In [50]:
LABEL_TRANSLATE = {0:'others', 1:'otolaryngology'}

In [51]:
def predict(model, x_data, y_data):
    # 텐서 하나
    # y_data의 차원을 확인하고 unsqueeze가 필요한지 확인
    if len(y_data.shape) == 1:  # 1D 텐서인 경우
        y_data = y_data.unsqueeze(1)  # (N,) -> (N, 1)
    
    y_data = y_data.float()  # float형으로 변환
    x_data = x_data.unsqueeze(1)
    with torch.no_grad():
        # 예측 수행
        pred = model(x_data)
        # 0.5 이상이면 1, 미만이면 0으로 변환
        pred_labels = torch.argmax((pred > 0.5).int())
        pred_labels = [LABEL_TRANSLATE[int(label)] for label in pred_labels.flatten()]
        real_labels = [LABEL_TRANSLATE[int(label)] for label in y_data.flatten()]

    return pred_labels, real_labels

In [53]:
lstm_model=torch.load('../../../LocalData/nlp/models/model_train_all_4_0.9889391654097537.pth', weights_only=False)

In [54]:
num_list=[3,5,7,8,11,13,17,900]
for num in num_list:    
    pred_label, real_label = predict(lstm_model, test_dataset.tensors[0][num], test_dataset.tensors[1][num])

    print(f"predict: {pred_label[0]}")
    print(f"real: {real_label[0]}\n")

predict: otolaryngology
real: otolaryngology

predict: otolaryngology
real: otolaryngology

predict: otolaryngology
real: otolaryngology

predict: otolaryngology
real: otolaryngology

predict: otolaryngology
real: otolaryngology

predict: otolaryngology
real: otolaryngology

predict: otolaryngology
real: otolaryngology

predict: otolaryngology
real: otolaryngology

